In [14]:
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
from small_concept_model.model import SmallConceptModel
from small_concept_model.inverter import PreNet, Inverter, get_encoder, get_gpt2_decoder
from small_concept_model.train import train_scm, train_inverter
from small_concept_model.data import get_bookcorpus_scm, get_bookcorpus_inverter
from small_concept_model.auto import build_scm, build_inverter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
scm_configs = {
    "d_model": 512,
    "d_embed": 384,
    "d_ff": 4 * 512,
    "n_heads": 4,
    "n_layers": 3,
    "dropout": 0.0,
    "max_seq_len": 128
}

prenet_configs = {
    "input_dim": 384,
    "output_dim": 768,
    "rank": 128,
    "prefix_len": 20,
}

train_configs = {
    "lr": 1e-3,
    "weight_decay": 1e-3,
    "batch_size": 32,
    "num_epochs": 1
}

In [5]:
encoder = get_encoder("paraphrase-multilingual-MiniLM-L12-v2")
prenet = PreNet(**prenet_configs).to(device)
decoder, tokenizer = get_gpt2_decoder()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [6]:
data = get_bookcorpus_inverter(
    encoder, tokenizer, max_target_len=64, embed_batch_size=32, sample=0.03, clean=True
)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:12<00:00, 75.81it/s]


In [12]:
train_inverter(prenet, decoder, tokenizer, data, **train_configs)

Epoch 1 [Train]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [06:55<00:00,  2.26it/s]

*** Epoch 0 Complete.  Avg Loss = 3.082486 ***


In [20]:
inverter = Inverter(prenet, decoder, tokenizer)

sample_text = "Hello, how are you doing?"
vec = encoder.encode(sample_text, convert_to_tensor=True)
inverter.invert(vec, max_len=30, temperature=0.8)

' are you alright? is there anything you are doing? is there anything you are doing wrong? is there something you are doing right now? is there'

---

In [4]:
encoder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device="cuda")
dataset = get_bookcorpus_scm(encoder, 32)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [10:46<00:00, 77.31it/s]


Train the model.

In [1]:
from datasets import load_dataset

data = load_dataset("francescoortame/bookcorpus-rand-1M", split="train")

/anaconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
texts = data["text"]

In [12]:
from tqdm import tqdm
from small_concept_model.utils import clean_text

clean_texts = []

for t in tqdm(texts, total=len(texts)):
    clean_texts.append(clean_text(t))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:07<00:00, 138347.40it/s]


In [ ]:


clean_t = [clean_text(t) for t in sample_texts]

for t in clean_t:
    print(t)

"Professor," evans said, "i have to go."
It made me uncomfortable.
Rex: short, bald, pale skin.
He'd called the glamour back in place.
Things were coming to a head, and we both knew it, although I am still reeling about how vlad got the jump on me.
It was the year of the accords, and all of downworld was abuzz about them and valentine's probable plans for disrupting them.
Hand it over.
"More horrible than you can even imagine."
He swings me around like i'm lighter than a feather and I laugh as my world spins.
I caught it without taking my eyes off his face.
Get up!
That made her feel a little better.
She buzzed the intercom and asked the receptionist to step inside.
You knew it was coming didnt you?
Are you staying or coming with us?
Neil let that pass with just a nod.
Without a word, the bartender took my mostly-empty mug and refilled it.
My arm flew out straight, palm up toward him and I cried, "don't!"
He grabbed my shoulders and forced me to face him.
She has just awakened, and all

In [36]:
from datasets import load_dataset

data = load_dataset("francescoortame/bookcorpus-sorted-100k16x", split="train")

In [35]:
train_scm(model, dataset, **train_configs)

Epoch [1/1]  Batch [100/3125]  Loss: 0.048322
Epoch [1/1]  Batch [200/3125]  Loss: 0.049153
Epoch [1/1]  Batch [300/3125]  Loss: 0.048909
Epoch [1/1]  Batch [400/3125]  Loss: 0.047952
Epoch [1/1]  Batch [500/3125]  Loss: 0.049336
Epoch [1/1]  Batch [600/3125]  Loss: 0.048689
Epoch [1/1]  Batch [700/3125]  Loss: 0.050626
Epoch [1/1]  Batch [800/3125]  Loss: 0.048821
Epoch [1/1]  Batch [900/3125]  Loss: 0.048079
Epoch [1/1]  Batch [1000/3125]  Loss: 0.050075
Epoch [1/1]  Batch [1100/3125]  Loss: 0.048651
Epoch [1/1]  Batch [1200/3125]  Loss: 0.048846
Epoch [1/1]  Batch [1300/3125]  Loss: 0.049641
Epoch [1/1]  Batch [1400/3125]  Loss: 0.049652
Epoch [1/1]  Batch [1500/3125]  Loss: 0.046929
Epoch [1/1]  Batch [1600/3125]  Loss: 0.048214
Epoch [1/1]  Batch [1700/3125]  Loss: 0.047378
Epoch [1/1]  Batch [1800/3125]  Loss: 0.047083
Epoch [1/1]  Batch [1900/3125]  Loss: 0.048429
Epoch [1/1]  Batch [2000/3125]  Loss: 0.048467
Epoch [1/1]  Batch [2100/3125]  Loss: 0.047782
Epoch [1/1]  Batch [22

## Pipeline

In [31]:
from small_concept_model.pipeline import Pipeline

model = model.to("cuda")
inverter = build_inverter("paraphrase_multilingual")
pipe = Pipeline(encoder, model, inverter)

In [39]:
texts = [
    'lexi stretched her arms up over her head as she waited patiently for jack to return and ravish her body .',
    'she heard the door open , and soft voices echoed down the hall toward her .',
]

pipe.generate(
    texts,
    n_future_steps = 5,
    sigma_noise = 0.0,
    temperature = 0.0,
    max_len = 30
)

[' she waited for her body to rest before she lifted her head and lifted her head up for her to rest . . . . she was exhausted . .',
 ' she heard the door open , and the door opened softly . the door opened , and the door opened softly . doors . doors . doors . doors .',
 " she said , and then she turned to the door , and the door opened . '' she said , and then she turned to the door , and the",
 " '' oh , she said , pointing at the door . '' and then she said , `` we should be talking about the door . '' . '' ''",
 " '' i should say , but i mean , he is a good liar . '' '' he replied . '' and then he said , `` oh , sorry",
 " '' i should say , but i mean , we are talking about the goddamned thing . '' '' he replied . '' i mean , we are",
 " '' i should say , but i mean , we are talking about the goddamned thing . '' '' he said . '' i mean , we are"]